In [1]:
import pandas as pd
from tensorflow.keras.layers import Input, Embedding, Dense, Dropout, Conv1D, GlobalMaxPool1D, concatenate
from konlpy.tag import Komoran
import pickle
import jpype
import tensorflow as tf
from tensorflow.keras.models import Model, load_model
from tensorflow.keras import preprocessing
from tensorflow.keras.layers import Input, Embedding, Dense, Dropout, Conv1D, GlobalMaxPool1D, concatenate

In [2]:
# 데이터 읽어오기
train_file = "finall_mbti_data.csv"
data = pd.read_csv(train_file)
data

,Unnamed: 0,text,type
0,0,엔프제 남자랑 연락하는데,0
1,1,내가 잘하고 있는걸까,0
2,2,아무리봐도 친구사이인데,0
3,3,이거 호감표현인가요,0
4,4,바람 펴본적 있거나 끌려본적 있음,0
...,...,...,...
390394,133432,쪼아,6
390395,133433,엣띱이다,6
390396,133434,이거친구들이보는것도잇던데,6
390397,133435,다들비슷하게나옴??,6


In [3]:
data.drop(['Unnamed: 0'], axis=1, inplace=True )
data

,text,type
0,엔프제 남자랑 연락하는데,0
1,내가 잘하고 있는걸까,0
2,아무리봐도 친구사이인데,0
3,이거 호감표현인가요,0
4,바람 펴본적 있거나 끌려본적 있음,0
...,...,...
390394,쪼아,6
390395,엣띱이다,6
390396,이거친구들이보는것도잇던데,6
390397,다들비슷하게나옴??,6


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 390399 entries, 0 to 390398
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    390242 non-null  object
 1   type    390399 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 6.0+ MB


In [5]:
# 결측치 제거
data = data.dropna(axis=0)
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 390242 entries, 0 to 390398
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    390242 non-null  object
 1   type    390242 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 8.9+ MB


In [6]:
data.sample(30)

,text,type
172605,근데 갤에 올리는 글,15
54600,돼지 혐오 문제점도 있긴함,4
160216,윤석열 대통령,11
231108,갤기장,8
151841,펌글 mbti와 혈액형 성격타입,11
291175,카페에서꼭쓰겠습니다,15
237513,잇팁아들 강제로 안으면,12
197754,엠티 검사할때 내가 되고 싶은걸로 검사해봤거든,9
112915,조언해줄사람이 주변에 없을 때 쓰는 방법,10
66679,신데 맘터 닫았어 씨발ㅋㅋㅋㅋㅋㅋ,4


In [7]:
class Preprocess:
    def __init__(self, word2index_dic='', userdic=None):
        # 단어 인덱스 사전 불러오기
        if(word2index_dic != ''):
            f = open(word2index_dic, "rb")
            self.word_index = pickle.load(f)
            f.close()
        else:
            self.word_index = None

        # 형태소 분석기 초기화
        self.komoran = Komoran(userdic=userdic)

        # 제외할 품사
        # 참조 : https://docs.komoran.kr/firststep/postypes.html
        # 관계언 제거, 기호 제거
        # 어미 제거
        # 접미사 제거
        self.exclusion_tags = [
            'JKS', 'JKC', 'JKG', 'JKO', 'JKB', 'JKV', 'JKQ',
            'JX', 'JC',
            'SS', 'SE'
        ]

    # 형태소 분석기 POS 태거
    def pos(self, sentence):
        jpype.attachThreadToJVM()
        return self.komoran.pos(sentence)

    # 불용어 제거 후, 필요한 품사 정보만 가져오기
    def get_keywords(self, pos, without_tag=False):
        f = lambda x: x in self.exclusion_tags
        word_list = []
        for p in pos:
            if f(p[1]) is False:
                word_list.append(p if without_tag is False else p[0])
        return word_list

    # 키워드를 단어 인덱스 시퀀스로 변환
    def get_wordidx_sequence(self, keywords):
        if self.word_index is None:
            return []

        
        w2i = []
        for word in keywords:
            try:
                w2i.append(self.word_index[word])
            except KeyError:
                # 해당 단어가 사전에 없는 경우, OOV 처리
                w2i.append(self.word_index['OOV'])
        return w2i

In [8]:
# MBTI 분류 모델 모듈
class MbtiModel:
    def __init__(self, model_name, proprocess):

        # MBTI 클래스 별 레이블
        self.labels = {0: "ENFJ", 1: "ENTJ", 2: "ESTJ", 3: "ESFJ", 4: "ENTP", 5: "ENFP", 6: "ESTP", 7: "ESFP", 8: "ISTJ", 9: "ISFJ",10: "INFJ", 11: "INTJ", 12: "ISTP", 13: "ISFP", 14: "INFP", 15: "INTP"}

        # 의도 분류 모델 불러오기
        self.model = load_model(model_name)

        # 챗봇 Preprocess 객체
        self.p = proprocess


    # 의도 클래스 예측
    def predict_class(self, query):
        # 형태소 분석
        pos = self.p.pos(query)

        # 문장내 키워드 추출(불용어 제거)
        keywords = self.p.get_keywords(pos, without_tag=True)
        sequences = [self.p.get_wordidx_sequence(keywords)]

        # 단어 시퀀스 벡터 크기
        MAX_SEQ_LEN = 20

        # 패딩처리
        padded_seqs = preprocessing.sequence.pad_sequences(sequences, maxlen=MAX_SEQ_LEN, padding='post')

        predict = self.model.predict(padded_seqs)
        predict_class = tf.math.argmax(predict, axis=1)
        return predict_class.numpy()[0]

In [9]:
from tensorflow.keras.callbacks import EarlyStopping
    
text = data['text'].tolist()
type = data['type'].tolist()

p = Preprocess(word2index_dic='dict.bin', userdic='mbti_dic.tsv')

# 단어 시퀀스 생성
sequences = []
for sentence in text:
    pos = p.pos(sentence)
    keywords = p.get_keywords(pos, without_tag=True)
    seq = p.get_wordidx_sequence(keywords)
    sequences.append(seq)

MAX_SEQ_LEN = 20

def GlobalParams():
    global MAX_SEQ_LEN

# 단어 인덱스 시퀀스 벡터 ○2
# 단어 시퀀스 벡터 크기
padded_seqs = preprocessing.sequence.pad_sequences(sequences, maxlen=MAX_SEQ_LEN, padding='post')

# (133437, 20)
print(padded_seqs.shape)
print(len(type)) #390242

C:\Users\jenny\AppData\Local\Temp/ipykernel_13608/2637904765.py:27: DeprecationWarning: jpype._core.attachThreadToJVM is deprecated, use java.lang.Thread.attach instead
  jpype.attachThreadToJVM()


(390242, 20)
390242


In [ ]:
# 학습용, 검증용, 테스트용 데이터셋 생성 ○3
# 학습셋:검증셋:테스트셋 = 7:2:1
ds = tf.data.Dataset.from_tensor_slices((padded_seqs, type))
ds = ds.shuffle(len(text))

train_size = int(len(padded_seqs) * 0.7)
val_size = int(len(padded_seqs) * 0.2)
test_size = int(len(padded_seqs) * 0.1)

train_ds = ds.take(train_size).batch(20)
val_ds = ds.skip(train_size).take(val_size).batch(20)
test_ds = ds.skip(train_size + val_size).take(test_size).batch(20)

# 하이퍼 파라미터 설정
dropout_prob = 0.5
EMB_SIZE = 128
EPOCH = 50
VOCAB_SIZE = len(p.word_index) + 1 #전체 단어 개수

# CNN 모델 정의  ○4
input_layer = Input(shape=(MAX_SEQ_LEN,))
embedding_layer = Embedding(VOCAB_SIZE, EMB_SIZE, input_length=MAX_SEQ_LEN)(input_layer)
dropout_emb = Dropout(rate=dropout_prob)(embedding_layer)
early_stopping = EarlyStopping(monitor = 'val_accuracy', patience = 3)

conv1 = Conv1D(
    filters=128,
    kernel_size=3,
    padding='valid',
    activation=tf.nn.relu)(dropout_emb)
pool1 = GlobalMaxPool1D()(conv1)

conv2 = Conv1D(
    filters=128,
    kernel_size=4,
    padding='valid',
    activation=tf.nn.relu)(dropout_emb)
pool2 = GlobalMaxPool1D()(conv2)

conv3 = Conv1D(
    filters=128,
    kernel_size=5,
    padding='valid',
    activation=tf.nn.relu)(dropout_emb)
pool3 = GlobalMaxPool1D()(conv3)

# 3,4,5gram 이후 합치기
concat = concatenate([pool1, pool2, pool3])

hidden = Dense(128, activation=tf.nn.relu)(concat)
dropout_hidden = Dropout(rate=dropout_prob)(hidden)
logits = Dense(16, name='logits')(dropout_hidden)
predictions = Dense(16, activation=tf.nn.softmax)(logits)


# 모델 생성  ○5
model = Model(inputs=input_layer, outputs=predictions)
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])


# 모델 학습 ○6
model.fit(train_ds, validation_data=val_ds, epochs=EPOCH, verbose=1, callbacks = [early_stopping])


# 모델 평가(테스트 데이터 셋 이용) ○7
loss, accuracy = model.evaluate(test_ds, verbose=1)
print('Accuracy: %f' % (accuracy * 100))
print('loss: %f' % (loss))


# 모델 저장  ○8
model.save('mbti_model.h5')

Epoch 1/50
13659/13659 [==============================] - 498s 36ms/step - loss: 2.2450 - accuracy: 0.2669 - val_loss: 2.1135 - val_accuracy: 0.3073
Epoch 2/50
13659/13659 [==============================] - 496s 36ms/step - loss: 2.1481 - accuracy: 0.2978 - val_loss: 2.0762 - val_accuracy: 0.3223
Epoch 3/50
13659/13659 [==============================] - 501s 37ms/step - loss: 2.1238 - accuracy: 0.3079 - val_loss: 2.0547 - val_accuracy: 0.3276
Epoch 4/50
13659/13659 [==============================] - 500s 37ms/step - loss: 2.1060 - accuracy: 0.3133 - val_loss: 2.0458 - val_accuracy: 0.3307
Epoch 5/50
13659/13659 [==============================] - 516s 38ms/step - loss: 2.0954 - accuracy: 0.3181 - val_loss: 2.0341 - val_accuracy: 0.3353
Epoch 6/50
13659/13659 [==============================] - 521s 38ms/step - loss: 2.0841 - accuracy: 0.3223 - val_loss: 2.0113 - val_accuracy: 0.3425
Epoch 7/50
13659/13659 [==============================] - 504s 37ms/step - loss: 2.0767 - accuracy: 0.3238

In [ ]:
mbti_labels = {0: "ENFJ", 1: "ENTJ", 2: "ESTJ", 3: "ESFJ", 4: "ENTP", 5: "ENFP", 6: "ESTP", 7: "ESFP", 8: "ISTJ", 9: "ISFJ",10: "INFJ", 11: "INTJ", 12: "ISTP", 13: "ISFP", 14: "INFP", 15: "INTP"}

# mbti 분류 모델 불러오기
model = load_model('mbti_model.h5')


p = Preprocess(word2index_dic='dict.bin',
               userdic='mbti_dic.tsv')

query = "나 여자친구 놀아써"

pos = p.pos(query)
keywords = p.get_keywords(pos, without_tag=True)
seq = p.get_wordidx_sequence(keywords)
sequences = [seq]

# 단어 시퀀스 벡터 크기
MAX_SEQ_LEN = 20

def GlobalParams():
    global MAX_SEQ_LEN

padded_seqs = preprocessing.sequence.pad_sequences(sequences, maxlen=MAX_SEQ_LEN, padding='post')

predict = model.predict(padded_seqs)
predict_class = tf.math.argmax(predict, axis=1)

print(query)
print("의도 예측 점수 : ", predict)
print("의도 예측 클래스 : ", predict_class.numpy())
print("의도  : ", mbti_labels[predict_class.numpy()[0]])

